https://adventofcode.com/2023/day/10

In [32]:
import math

import networkx as nx

In [33]:
with open("data/10.txt") as fh:
    data = fh.read()

In [34]:
testdata = """\
..F7.
.FJ|.
SJ.L7
|F--J
LJ...
"""

In [35]:
def parse_input(data):
    G = nx.Graph()
    spos = None
    for row, line in enumerate(data.splitlines()):
        for col, char in enumerate(line):
            pos = complex(col, -row)
            if char == "S":
                spos = pos
            else:
                G.add_node(pos, t=char)
    return G, spos

In [36]:
pipe_join_lookup = {
    "|": {1j, -1j},
    "-": {1, -1},
    "L": {1j, 1},
    "J": {1j, -1},
    "7": {-1j, -1},
    "F": {-1j, 1},
    "S": {1, -1, 1j, -1j},
    ".": set(),
}

In [40]:
def join_pipe_segments(G, spos):
    for pos, atts in G.nodes(data=True):
        tdirs = pipe_join_lookup[atts["t"]]
        for tdir in tdirs:
            pos_2 = pos + tdir
            if pos_2 in G:
                t_2 = G.nodes[pos_2]["t"]
                tdirs_2 = pipe_join_lookup[t_2]
                if -tdir in tdirs_2:
                    G.add_edge(pos, pos_2)
    G.add_node(spos, t="S")
    for drxn in [1, -1, 1j, -1j]:
        pos_2 = spos + drxn
        if pos_2 in G:
            t_2 = G.nodes[pos_2]["t"]
            tdirs_2 = pipe_join_lookup[t_2]
            if -drxn in tdirs_2:
                G.add_edge(spos, pos_2)

In [41]:
def half_loop(data):
    G, spos = parse_input(data)
    join_pipe_segments(G, spos)
    loop = nx.find_cycle(G, spos)
    return len(loop) // 2

In [42]:
half_loop(testdata)

8

In [43]:
half_loop(data)

6875

### Part 2

In [10]:
td1 = """\
...........
.S-------7.
.|F-----7|.
.||.....||.
.||.....||.
.|L-7.F-J|.
.|..|.|..|.
.L--J.L--J.
...........
"""

td2 = """\
.F----7F7F7F7F-7....
.|F--7||||||||FJ....
.||.FJ||||||||L7....
FJL7L7LJLJ||LJ.L-7..
L--J.L7...LJS7F-7L7.
....F-J..F7FJ|L7L7L7
....L7.F7||L7|.L7L7|
.....|FJLJ|FJ|F7|.LJ
....FJL-7.||.||||...
....L---J.LJ.LJLJ...
"""

td3 = """\
FF7FSF7F7F7F7F7F---7
L|LJ||||||||||||F--J
FL-7LJLJ||||||LJL-77
F--JF--7||LJLJ7F7FJ-
L---JF-JLJ.||-FJLJJ7
|F|F-JF---7F7-L7L|7|
|FFJF7L7F-JF7|JL---7
7-L-JL7||F7|L7F-7F7|
L.L7LFJ|||||FJL7||LJ
L7JLJL-JLJLJL--JLJ.L"""

In [11]:
def count_inside_tiles(data):
    G, spos = parse_input(data)
    join_pipe_segments(G, spos)
    loop = nx.find_cycle(G, spos)

    node_next = {}
    node_prev = {}
    for a, b in loop:
        node_next[a] = b
        node_prev[b] = a

    for node, nxt in node_next.items():
        prv = node_prev[node]
        set_directions(G, node, nxt, prv)
    
    inside_rotator = get_inside_rotator(G, node_next)
    
    for node in node_next:
        mark_loop_node_neighbors(G, node, inside_rotator)
    
    connect_components(G)
    inside_count = 0
    for cc in nx.connected_components(G):
        for node in cc:
            if G.nodes[node].get("inside"):
                inside_count += len(cc)
                break 
    return inside_count


def connect_components(G):
    # connect up all nodes without crossing loop
    for node in G:
        node_d = G.nodes.get(node)
        if node is None:
            continue
        if node_d.get("onloop"):
            continue
        for d in [1, 0 + 1j, -1, 0 - 1j]:
            nabe = node + d
            if nabe not in G:
                continue
            nabe_d = G.nodes[nabe]
            if nabe_d.get("onloop"):
                continue
            G.add_edge(node, nabe)


def mark_loop_node_neighbors(G, node, inside_rotator):
    node_d = G.nodes[node]
    inside_drxns = {node_d["drxn"] * inside_rotator, node_d["prev_drxn"] * inside_rotator}
    for d in [1, 0 + 1j, -1, 0 - 1j]:
        nabe = node + d
        if nabe not in G:
            continue
        nabe_d = G.nodes[nabe]
        if nabe_d.get("onloop"):
            continue
        if d in inside_drxns:
            nabe_d["inside"] = True
            

def get_inside_rotator(G, node_next):    
    leftmost = math.inf + 0j
    for node in node_next:
            if node.real < leftmost.real:
                leftmost = node
    leftmost_d = G.nodes[leftmost]
    leftmost_drxns = leftmost_d["drxn"], leftmost_d["prev_drxn"]
    if 1j in leftmost_drxns:
        inside_rotator = 0-1j
    elif -1j in leftmost_drxns:
        inside_rotator = 0+1j
    else:
        raise ValueError("Can't happen")
    return inside_rotator
    

def set_directions(G, node, nxt, prv):
    D = G.nodes[node]
    D["onloop"] = True
    D["drxn"] = nxt - node
    D["prev_drxn"] = node - prv    

In [12]:
count_inside_tiles(td1)

4

In [13]:
count_inside_tiles(td2)

8

In [14]:
count_inside_tiles(td3)

10

In [15]:
count_inside_tiles(data)

471